# Mark Galperin - Final Project
# Due December 8, 2020

This is my python notebook for the final project for ME 314! Wahoo

**Imports**

In [2]:
import sympy as sym 
from sympy.abc import t
print(sym.__version__)

1.6.2


In [3]:
#other imports...
from sympy import Function, Matrix, Eq, symbols
import numpy as np
import matplotlib.pyplot as plt

**Helper Functions**

In [4]:
def get_SE3_planar(R_th,p):
    """
    This function assembles planar homogenous transforms in SE(3) given either a rotation and a translation,
    or an R matrix and a translation.
    
    Takes...
        R_th: This input can be either a sympy matrix (at least 2x2) that is the rotation matrix for
            planar rotation about z, OR it can be a numerical value for the angle theta rotated about z
        
        p: a 3x1 (or 2x1) sympy matrix that is a homogenous coordinate vector for an x, y translation.
            p=0 is an acceptable input, shorthand for p = Matrix([0,0,0])
        
    Returns...
        g_SE3: a 3x3 sympy matrix that is a planar homogenous transform in SE(3)
    """
    
    #allowing p=0 as an input...
    if p==0:
        p = Matrix([0,0,0])
    
    #assembling the matrix...
    if type(R_th).__name__ == 'MutableDenseMatrix':
        #given R_th is a matrix in SE(2) at least... 
        g_SE3 = Matrix([[R_th[0,0], R_th[0,1], 0, p[0]],
                  [R_th[1,0], R_th[1,1], 0, p[1]],
                  [0, 0, 1, 0],
                  [0, 0, 0, 1]])
    else:   
        #if R_or_th is a numerical value...
        g_SE3 = Matrix([[sym.cos(R_th), -sym.sin(R_th), 0, p[0]],
                      [sym.sin(R_th), sym.cos(R_th), 0, p[1]],
                      [0, 0, 1, 0],
                      [0, 0, 0, 1]])
    
    return g_SE3

In [5]:
def get_SE3_planar_np(R_th,p):
    """
    This function assembles planar homogenous transforms in SE(3) given either a rotation and a translation,
    or an R matrix and a translation.
    
    Takes...
        R_th: This input can be either a sympy matrix (at least 2x2) that is the rotation matrix for
            planar rotation about z, OR it can be a numerical value for the angle theta rotated about z
        
        p: a 3x1 (or 2x1) sympy matrix that is a homogenous coordinate vector for an x, y translation.
            p=0 is an acceptable input, shorthand for p = np.array([0,0,0])
        
    Returns...
        g_SE3: a 3x3 numpy array that is a planar homogenous transform in SE(3)
    """
    
    #allowing p=0 as an input...
    if p==0:
        p = np.array([0,0,0])
    
    #assembling the matrix...
    if type(R_th).__name__ == 'ndarray':
        #given R_th is a matrix in SE(2) at least... 
        g_SE3 = np.array([[R_th[0][0], R_th[0][1], 0, p[0]],
                  [R_th[1][0], R_th[1][1], 0, p[1]],
                  [0, 0, 1, 0],
                  [0, 0, 0, 1]])
    else:   
        #if R_or_th is a numerical value...
        g_SE3 = np.array([[np.cos(R_th), -np.sin(R_th), 0, p[0]],
                      [np.sin(R_th), np.cos(R_th), 0, p[1]],
                      [0, 0, 1, 0],
                      [0, 0, 0, 1]])
    
    return g_SE3

In [6]:
def hat(w):
    w_hat = Matrix([[0,    -w[2], w[1] ],
                    [w[2],   0,  -w[0] ],
                    [-w[1], w[0],   0  ]])
    return w_hat

In [7]:
def unhat_3x3(w_hat):
    return Matrix([w_hat[2,1],w_hat[0,2],w_hat[1,0]])

In [ ]:
def unhat_4x4(w_hat):
    return Matrix([w_hat[2,1],w_hat[0,2],w_hat[1,0]])

In [8]:
def g_inv(g):
    """
    This function calculates the inverse of a transformation g, in (planar) SE(3). 
    
    Dependencies:
        get_SE3_planar(th,p): Generates a planar homogenous transform in SE(3). Used for output.
    
    Takes...
        g: a 3x3 sympy matrix in SE(3)
        
    Returns...
        g_out: a 3x3 sympy matrix in SE(3) that is the "inverse" of g
    """
    #dissecting the input transformation g
    R = g[0:3,0:3]
    p = Matrix([g[0,3],g[1,3],1])
    
    RT = R.T
    p_inv = -RT*p
    
    return get_SE3_planar(RT,p_inv)

In [9]:
def g_dot(g,q_th):
    """
    This function gives the time-derivative of g
    g_dot = [R_dot p_dot
             0     1    ]
    ...where R_dot = Rw = wR, and w = [0 0 d(th)/dt]
    
    TODO - more docstring
    """
    
    w = Matrix([0,0,q_th.diff(t)])
    
    #R_dot...
    R = g[0:3,0:3]
    R_dot = R*hat(w)
    
    #p_dot...
    p = g[0:3,3]
    p_dot = p.diff(t)
    
    #generating the transform...
    return get_SE3_planar(R_dot,p_dot)

In [10]:
def get_Vb(g,q_th,select='Vb'):
    
    #TODO: selection dictionary
    
    Vhat = g_inv(g)*g_dot(g,q_th)
    w_hat = Vhat[0:3,0:3]
    v = Vhat[0:3,3]
    w = unhat(w_hat)
    Vb = v.col_join(w)
    return Vb

In [6]:
a = np.array([[1,2,3],[4,5,6],[7,8,9]])
type(a).__name__

'ndarray'